# azure open ai 使用入门

## 推荐阅读

> - [Azure OpenAI 使用说明文档](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/)

## 基本概念

- subscription：订阅，类似于一个租户的概念，目前只能开3个订阅
- region：区域，资源部署的位置，目前只支持east us, south central us和west europe, 其中west europe没有gpt-35-turbo的模型
- resource group: 资源组，每个订阅下可以设置多个资源组，目前openai的资源统一放在 AIGC的资源组中
- resource：资源，一个openai服务就是一个资源，每个资源提供一个访问的endpoint
- deployment / model: 模型，在每个资源内部，可以部署不同种类的openai模型，也可以进行fine-tune训练部署自己的模型。

## 模型

[Azure OpenAI 服务配额和限制](https://learn.microsoft.com/zh-cn/azure/cognitive-services/openai/quotas-limits)

## 使用入门实例

参考：[openai/openai-python](https://github.com/openai/openai-python)

In [ ]:
# 下载依赖
!pip install -q python-dotenv
!pip install -q openai
!pip install -q azure_identity

In [ ]:
# 读取azure账号配置

import os
from dotenv import load_dotenv, find_dotenv

# 加载.env 环境变量
_ = load_dotenv(find_dotenv())

# 读取系统中的环境变量
client_id  = os.getenv('client_id')
client_secret = os.getenv('client_secret')
tenant_id = os.getenv('tenant_id')
endpoint = os.getenv('endpoint')
print(endpoint)

In [ ]:
from azure.identity import ClientSecretCredential
 
# ref: https://github.com/openai/openai-python
sp_credential = ClientSecretCredential(
    client_id=client_id,
    client_secret=client_secret,
    tenant_id=tenant_id)
token = sp_credential.get_token("https://cognitiveservices.azure.com/.default")
print(token)

In [ ]:
import openai

# use aad token
openai.api_type = "azure_ad"
openai.api_key = token.token
openai.api_base = endpoint
openai.api_version = "2023-03-15-preview"

# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    # messages = [{"role":"system","content":"who are you?"},{"role": "user", "content": prompt}]
    messages = [{"role": "user", "content": prompt}]
    response =  openai.ChatCompletion.create(
      engine=model,
      messages = messages,
     temperature=0,
      max_tokens=250,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

In [ ]:
prompt = f""" 你是谁
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)